In [ ]:
%matplotlib inline
##%matplotlib nbagg

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, rc, animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

plt.switch_backend("TkAgg")
plt.rcParams['animation.ffmpeg_path'] = "/usr/local/bin/ffmpeg"

In [ ]:
## Define functions used to generate data of boundary.
def zero(x):
    return 0
def slope_one(x, b=0):
    return x+b
def slope_minus_one(x, b=19):
    return b-x
def slope_one_third(x, b=0):
    return x/3 + b
def sin(x):
    return np.sin(x*12/180*np.pi)
def cos(x):
    return np.cos(x*12/180*np.pi)

In [ ]:
## Define a "plate" class.
## The plate class contains number of sides, width, length and a 2D numpy array of grid points.
## Number of sides decides shape of a plate.
## For example, sides=4 represents rectangle, sides=0 represents circle.
## Width and length determine size of a plate.
## The 2D numpy array of grid points records the value at every grid point, and the index 
## represents the coordinate of each data on the xy-plane.

class plate:
    def __init__(self, sides=4, width=10, length=10):
        self.sides = sides
        self.w = width
        self.l = length
    
    def init_points(self):
        if self.sides == 4:
            self.dots = np.zeros(shape=(self.w, self.l))
        elif self.sides == 0:
            self.dots = np.zeros(shape=(self.w, 30))
    
    def set_BC(self, f_xu=zero, f_xl=zero, f_yu=zero, f_yl=zero):
        if self.sides == 4:
            x_max = self.w
            y_max = self.l
            for i in range(x_max):
                self.dots[i][0] = f_xu(i)
            for i in range(x_max):
                self.dots[i][y_max-1] = f_xl(i)
            for i in range(y_max):
                self.dots[0][i] = f_yu(i)
            for i in range(y_max):
                self.dots[x_max-1][i] = f_yl(i)
        elif self.sides == 0:
            for i in range(30):
                self.dots[-1][i] = f_xu(i)
    
    def average(self, error=0.001):
        is_balance = True
        if self.sides == 4:
            row = self.w
            col = self.l
            for i in range(row-2):
                i = i + 1
                for j in range(col-2):
                    j = j + 1
                    tmp = (self.dots[i-1][j] + self.dots[i+1][j] + self.dots[i][j-1] + self.dots[i][j+1]) / 4
                    if abs(tmp - self.dots[i][j]) > error:
                        is_balance = False
        elif self.sides == 0:
            r = self.w
            for i in range(r-1):
                for j in range(30):
                    if i == 0:
                        tmp = (self.dots[i+1][j] + self.dots[i][(j+1)%30] + self.dots[i][j-1]) / 4
                    else:
                        tmp = (self.dots[i-1][j] + self.dots[i+1][j] + self.dots[i][(j+1)%30] + self.dots[i][j-1]) / 4
                    if abs(tmp - self.dots[i][j]) > error:
                        is_balace = False        

In [ ]:
## Define a "draw_plate" function, which will be called by "FuncAnimation" method to draw the graph dynamically.

def draw_plate(num, plate, error):
    if plate.sides == 4:
        xs = np.arange(1, plate.w+1)
        ys = np.arange(1, plate.l+1)
        Y, X = np.meshgrid(ys, xs)
    elif plate.sides == 0:
        rs = np.arange(0, plate.w)
        thetas = np.linspace(0, 2*np.pi, 30)
        X = np.outer(rs, np.cos(thetas))
        Y = np.outer(rs, np.sin(thetas))
    ax.clear()
    plate.average(error)
    surf = ax.plot_surface(X, Y, plate.dots, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0)

In [ ]:
## Get variables needed to construct a plate object and compute the average value.

print("Sides: 0 = circle, 4 = rectangle.")
print("Width: Width of a rectangle, or radius of a circle.")
print("Length: Length of a rectangle. Please input 0 when using circular boundary.")
print("Error: Tolerance.\n\n")
sides = int(input("Sides:"))
width = int(input("Width:"))
length = int(input("Length:"))
error = float(input("Error:"))

## Construct a plate, initialize the value of grid points in the plate, and set boundary condition.

pl = plate(sides, width, length)
pl.init_points()
pl.set_BC(cos, cos, cos, cos)

## Generate a 3D figure.
fig = plt.figure()
ax = fig.gca(projection="3d")

## Draw the graph.

ani = animation.FuncAnimation(fig, draw_plate, fargs=(pl, error), interval=50)
HTML(ani.to_html5_video())
rc("animation", html="html5")
ani
##plt.show()